In [2]:
'" asdf'.strip('"')

' asdf'

In [1]:
cat .gitignore

# Compiled source #
###################
*.com
*.class
*.dll
*.exe
*.o
*.so

# Packages #
############
# it's better to unpack these files and commit the raw source
# git has its own built in compression methods
*.7z
*.dmg
*.gz
*.iso
*.jar
*.rar
*.tar
*.zip

# Logs and databases #
######################
*.log
*.sql
*.sqlite

# OS generated files #
######################
.DS_Store
.DS_Store?
._*
.Spotlight-V100
.Trashes
ehthumbs.db
Thumbs.db
telegram_api.key
translation_api.key

In [7]:
!echo trnsl.1.1.20161213T195505Z.da017ed804634d3b.ace1de9774f00a39a770fb3fe585d457ab9dfde > translation_api.key

In [9]:
cat telegram_api.key

392437358:AAETwo_1h2yDxjtWczY59svY7YylYcZKZak


In [18]:
'{0} {0}'.format('weee')

'weee weee'

In [33]:
TELEGRAM_KEY_FILE = 'telegram_api.key'
TRANSLATION_KEY_FILE = 'translation_api.key'


def read_key_file(service):
    if service == 'Telegram':
        key_file = TELEGRAM_KEY_FILE
    elif service == 'Translation':
        key_file = TRANSLATION_KEY_FILE
    else:
        raise Exception('Service name is not correct, cannot read keys')
        
    try:
        with open(key_file) as f:
            read_token = f.readline().strip()
    except FileNotFoundError:
        raise Exception('{0} key file not found, please place your {0} API token to {1} file'.format(service, key_file))
    
    return read_token
    
TELEGRAM_TOKEN = read_key_file('Telegram')
TRANSLATION_TOKEN = read_key_file('Translation')

In [34]:
TRANSLATION_TOKEN

'trnsl.1.1.20161213T195505Z.da017ed804634d3b.ace1de9774f00a39a770fb3fe585d457ab9dfde'

In [ ]:
open('translation_api.key')

In [1]:
from requests import get, post
#requests - даёт пользоваться http Без особых знаний
import urllib
import urllib.parse
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML

TODO: 
* выводить табличкой
* добавить ссылку на сайт

In [2]:
def translate(city_name, final_lang="ru"):
    api_key = 'trnsl.1.1.20161213T195505Z.da017ed804634d3b.ace1de9774f00a39a770fb3fe585d457ab9dfde'
    template = 'https://translate.yandex.net/api/v1.5/tr.json/translate?&key={}c&text={}&lang={}'
    url = template.format(api_key,city_name, final_lang)
    json_request = get(url).json()
    return json_request['text'][0]

In [3]:
def get_genre(nouns):
    for noun in nouns:
        print()
        print('Translation: {}'.format(translate(noun,final_lang='fr-ru')))
        template = 'http://www.linternaute.com/dictionnaire/fr/definition/{}/'
        try:
            url = template.format(noun)
            req = get(url)

            soup = BeautifulSoup(req.text, "lxml")
            print (soup.select('.dico_title_1')[0].text_content())
            display(HTML(str(soup.select('.dico_title_1')[0])))
            display(HTML(str(soup.select('.dico_title_definition')[0])))
        except:
            print('Failed')

In [67]:
def get_genre(nouns):
    for noun in nouns:
        print('{} means {}'.format(noun, translate(noun,final_lang='fr-ru')))
        template = 'http://www.linternaute.com/dictionnaire/fr/definition/{}/'
        try:
            url = template.format(noun)
            req = get(url)

            soup = BeautifulSoup(req.text, "lxml")
            word = soup.select('.dico_title_1')[0]
            word_str = word.get_text()
            genre = soup.select('.dico_title_definition')[0]
            genre_str = genre.get_text().replace('\n', '').replace('\t', '').strip()
            print('{} is {}'.format(word_str, genre_str))
        except:
            print('Failed')
    return genre

In [68]:
genre = get_genre(['chat'])

chat means кошка
chat is nom, nom masculin


In [56]:
genre.get_text().replace('\n', '').replace('\t', '').strip()

'nom, nom masculin'

In [4]:
def get_conjugation(verbs, time):
    print('{} forms for:'.format(time))
    for verb in verbs:
        print()
        print(verb)
        print(translate(verb,final_lang='fr-en'))
        if (verb.count(' ') > 0):
            verb = verb.replace(' ', '_')
        template = 'http://la-conjugaison.nouvelobs.com/du/verbe/{}.php'
        url = template.format(verb)
        req = get(url)

        assert req.status_code == 200, 'request failed'
        soup = BeautifulSoup(req.text, "lxml")
        display(HTML(str(soup.select('.bloc b')[0])))
        tempstab = soup.findAll('div', attrs={'class' : 'tempstab'})
        hey = []
        for temp in tempstab:
            if (temp.findAll('img', attrs={'alt': time}) != []):
                hey.append(temp)
        
        if (len(hey) == 0):
            print('Conjugation not found for verb {}, try:'.format(verb))
            print(url)
        else:
            hey = hey[0].find('div', attrs={'class': 'tempscorps'})
            display(HTML(str(hey)))

In [29]:
sss = 'partir is verbe'

In [31]:
'versb' in sss 

False

In [5]:
def get_conjugation(verb):
#     print('{} forms for:'.format(time))
    times = ['Présent', 'Passé composé', 'Imparfait', 'Futur simple']
    
    if (verb.count(' ') > 0):
        verb = verb.replace(' ', '_')
    template = 'http://la-conjugaison.nouvelobs.com/du/verbe/{}.php'
    url = template.format(verb)
    req = get(url)

    assert req.status_code == 200, 'request failed'
    soup = BeautifulSoup(req.text, "lxml")
    tempstab = soup.findAll('div', attrs={'class' : 'tempstab'})
    result = ''
    for time in times:
        hey = []
        for temp in tempstab:
            if (temp.findAll('img', attrs={'alt': time}) != []):
                hey.append(temp)

        if (len(hey) == 0):
            print('Conjugation not found for verb {}, try:'.format(verb))
            print(url)
        else:
            hey = hey[0].find('div', attrs={'class': 'tempscorps'})
#             display(HTML(str(hey)))
            result += '{} : {}\n'.format(time, hey.get_text())
    return result

In [48]:
sup = get_conjugation('sortir')

In [45]:
pronouns = ['tu ', 'il ', 'elle ', 'nous ', 'vous ', 'ils ']
test = 'tu '
new_sup = sup

In [46]:
for test in pronouns:
    index = 0
    counter = 0
    while index != -1:
        counter += 1
        if counter > 20:
            print('wrong')
            break
        index = new_sup.find(test)
        if index != -1:
            new_sup = new_sup[:index] + ' ' + test[0].upper() + new_sup[index+1:]

In [47]:
new_sup

'Présent : je pars Tu pars Il part Nous partons Vous partez Ils partent\nPassé composé : je suis parti Tu es parti Il est parti Nous sommes partis Vous êtes partis Ils sont partis\nImparfait : je partais Tu partais Il partait Nous partions Vous partiez Ils partaient\nFutur simple : je partirai Tu partiras Il partira Nous partirons Vous partirez Ils partiront\n'

In [ ]:

line[:index] + 'Fu ' + line[index:]

In [46]:
import re

In [50]:
re.split('x*', 'axbc')

C:\Anaconda3\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


['a', 'bc']

In [18]:
get_conjugation(['partir','etre', 'lire', 'ecouter', 'attendre', 'etre', 'rever', 'se reunir', 'discuter', 'manger', 'lancer', 'etre', 'croire', 'marcher', 'reflechir', 'etre', 'commencer', 'faire', 'geler', 'avoir'],'Imparfait')

Imparfait forms for:

partir
from



etre
be



lire
read



ecouter
listen



attendre
wait


...


etre
be



commencer
start



faire
do



geler
freeze



avoir
have


In [137]:
get_conjugation(['aller', 'eteindre', 'ecrire', 'traduire', 'avoir','avoir','obeir', 'laver','ranger', 'conduire', 'travailler', 'applaudir', 'voir','rougir'], 'Imparfait')

Imparfait forms for:

aller
go



eteindre
turn off



ecrire
write



traduire
translate



avoir
have


...


conduire
lead



travailler
work



applaudir
cheer



voir
see



rougir
blush


In [134]:
translate('Ce etait un matin du octobre, les arbres commencaient a perdre leurs feuilles. Il faisait froid, il gelait, il y avait du vent.',final_lang = 'en')

'This was a morning of October, the trees started losing their leaves. It was cold, it was freezing, it was windy.'

In [145]:
translate('Nous avions peur de notre pere et nous lui obéissions. pendant ce temps, nous rangions les affaires. a ce moment notre voisin a appele au secours', 'en')

'We were afraid of our father and obey him. during this time, we rangions the business. at this time our neighbor has called to the aid'

In [181]:
get_conjugation(['briller'],'Passé composé')
# get_conjugation(['eplucher'],'Passé composé')

Passé composé forms for:

briller
shine


In [6]:
get_conjugation(['sourire'],'Présent')
get_conjugation(['sourire'],'Passé composé')

# get_conjugation(['etre'],'Imparfait')

Présent forms for:

sourire
smile


Passé composé forms for:

sourire
smile


In [184]:
translate('Tous les jours, Victor jouais aux echecs', 'en')
translate('Il faisait nuit, les etoiles brillaient dans le ciel', 'en')

'It was night, the stars shining in the sky'

In [6]:
etre_verbs = ['Devenir', 'Revenir', 'Monter', 'Rester', 'Sortir', 'Venir', 'Aller',
       'Naitre', 'Descendre', 'Entrer', 'Retourner', 'Tomber', 'Rentrer', 'Arriver', 'Mourir', 'Partir']

In [20]:
get_conjugation(etre_verbs, 'Passé composé')

Passé composé forms for:

Devenir
Become



Revenir
Return



Monter
Mount



Rester
Stay



Sortir
Out


...


Tomber
Fall



Rentrer
Return



Arriver
Getting



Mourir
Die



Partir
From


* 1я группа: monter, rester, entrer, retourner, tomber, rentrer, arriver, aller(3я, но в passe compose также): -er -> é
* 3я группа:
    * devenir, revenir, venir: -enir -> enu
    * sortir, partir -> sorti, parti
    * naitre (né), mourir(mort), descendre (descendu) - не похожи

In [21]:
len(etre_verbs)

16

In [5]:
def get_passe_and_imparfait(verbs):
    for verb in verbs:
        print()
        print(verb)
        print(translate(verb,final_lang='fr-en'))
        if (verb.count(' ') > 0):
            verb = verb.replace(' ', '_')
        template = 'http://la-conjugaison.nouvelobs.com/du/verbe/{}.php'
        url = template.format(verb)
        req = get(url)

        assert req.status_code == 200, 'request failed'
        soup = BeautifulSoup(req.text, "lxml")
        display(HTML('nous: ' + str(soup.select('.tempstab:nth-child(11) b:nth-child(7)')[0])))
        display(HTML('Compose: ' + str(soup.select('.tempstab:nth-child(12) b:nth-child(1)')[0])))

In [6]:
get_passe_and_imparfait(['etre'])


etre
be


NotImplementedError: Only the following pseudo-classes are implemented: nth-of-type.

In [13]:
url = 'http://la-conjugaison.nouvelobs.com/du/verbe/etre.php'
req = get(url)

assert req.status_code == 200, 'request failed'
soup = BeautifulSoup(req.text, "lxml")
display(HTML(str(soup.select('.tempstab:nth-child(12) .tempscorps'))))
# display(HTML('Compose: ' + str(soup.select('.tempstab:nth-child(12) b:nth-child(1)')[0])))